In [1]:
from floodlight.io import dfl
from floodlight.io import opta
import numpy as np
import pandas as pd
import fileinput
import os
import csv
import glob
from lxml import etree
import pickle

In [2]:
# f24-22-2016-861416
# needed to check if opta reports an actual red card (or includes a qualifier signaling a correction of the decision for example)

Events = opta.read_event_data_xml('EventData/f24-22-2016-861416-eventdetails.xml')[0]      # read the events

data = Events['HT2']['Home'].events
cards = data[data.eID == 17]
cards

,eID,gameclock,pID,outcome,timestamp,minute,second,at_x,at_y,qualifier
236,17,1342.0,13010.0,1,2016-09-17 15:54:38.106000+00:00,22,22,0.0,0.0,"{31: None, 13: '243'}"
277,17,1671.5,53643.0,1,2016-09-17 16:00:07.606000+00:00,27,51,0.0,0.0,"{184: None, 31: None}"
384,17,2474.5,57500.0,1,2016-09-17 16:13:30.606000+00:00,41,14,0.0,0.0,"{13: '243', 31: None}"
389,17,2617.0,NaN,1,2016-09-17 16:15:53.106000+00:00,43,37,0.0,0.0,"{283: '39650', 33: None, 159: None}"


## Overview of games in event data files

In [3]:
# relevant information collection lists

matchdays_numeric = []    # number of the matchday
opta_mns = []    # opta name of the match
home_teams =  []    # home teams
away_teams =  []    # away teams
dates = []           # date of the match
 
halfs = ['HT1', 'HT2']     # halfs for loop
teams = ['Home', 'Away']   # teams for loop
bools_c = []                # store bools of cards on match level
bools_r = []                # store bools of retired on match level

retired = []             # store bools of retired one for each match
red_card = []            # store bools of red_cards one for each match

dirHome1 = []
dirHome2 = []
dirAway1 = []
dirAway2 = []



ROOT = os.getcwd() # returns current working directory of a process.

ev_data = (r"Eventdata") # creates a raw string which treats backslash (\) as a literal character. 

for dir in os.listdir(ev_data):
    if dir.startswith('Spieltag'):
        md = dir
        ST = md.replace('Spieltag ', '')# get the numerical matchday
        loc = os.path.join(ev_data, dir) 
        # print(loc)
        for eve_file in os.listdir(loc):
            # print(eve_file)
            if eve_file.startswith('f'):
                opta_mn = eve_file.replace('-eventdetails.xml', '')
                tree = etree.parse(str('/'.join([loc, eve_file])))
                root = tree.getroot()
                matchinfo = root.xpath("Game")[0].attrib
                matchdays_numeric.append(ST)
                opta_mns.append(opta_mn)
                home_teams.append(matchinfo['home_team_name'])
                away_teams.append(matchinfo['away_team_name'])
                dates.append(matchinfo['game_date'])
                
                Events = opta.read_event_data_xml(str('/'.join([loc, eve_file])))[0]      # read the events
                halfs = ['HT1', 'HT2']
                teams = ['Home', 'Away']
                bools_c = []
                bools_r = []
                
                
                for h in halfs:
                    for t in teams:
                        e_data = Events[h][t].events

                        # any one being subbed off due to injury when no subs are left in this match?
                        ret = e_data[e_data.eID == 20]
                        if len(ret) > 0:
                            bools_r.append(True)
                        else:
                            bools_r.append(False)

                        # any red cards or second yellows in this match?
                        cards = e_data[e_data.eID == 17]
                        bools_c.append(any(cards.qualifier.str.contains('32:')) or any(cards.qualifier.str.contains('33:')))
                        
                        # match direction for that team
                        di = e_data[e_data.eID == 32]
                        if len(di) > 1:
                            print('More than one start for that half')
                        else: 
                            if any(di.qualifier.str.contains("'Right to Left'")):
                                direction = 'Right to Left'
                            elif any(di.qualifier.str.contains("'Left to Right'")):
                                direction = 'Left to Right'
                            else:
                                direction = 'No direction found'
                            if h == 'HT1':
                                if t == 'Home':
                                    dirHome1.append(direction)
                                elif t == 'Away':
                                    dirAway1.append(direction)
                            elif h == 'HT2':
                                if t == 'Home':
                                    dirHome2.append(direction)
                                elif t == 'Away':
                                    dirAway2.append(direction)
                                    

                red_card.append(any(bools_c))    # in any of the 2 teams and two halves a red card
                retired.append(any(bools_r))       # in any of the 2 teams and two halves a retirement
 

                
                
                
opta_game_overview = pd.DataFrame(list(zip(dates, home_teams, away_teams, matchdays_numeric, opta_mns, red_card, retired, dirHome1, dirHome2, dirAway1, dirAway2)),
                             columns=['Opta_Date', 'Opta_Home', 'Opta_Away', 'Opta_Matchday', 'Opta_Name', 'Red_card', 'Retired', 'Direction_Home_HT1', 'Direction_Home_HT2', 'Direction_Away_HT1', 'Direction_Away_HT2'])
opta_game_overview

# events missing for one game on matchday 2

,Opta_Date,Opta_Home,Opta_Away,Opta_Matchday,Opta_Name,Red_card,Retired,Direction_Home_HT1,Direction_Home_HT2,Direction_Away_HT1,Direction_Away_HT2
0,2016-08-26T19:30:00,FC Bayern München,SV Werder Bremen,1,f24-22-2016-861390,False,False,Right to Left,Left to Right,Left to Right,Right to Left
1,2016-08-27T14:30:00,Borussia Dortmund,1. FSV Mainz 05,1,f24-22-2016-861391,False,False,Left to Right,Right to Left,Right to Left,Left to Right
2,2016-08-27T17:30:00,Borussia Mönchengladbach,Bayer 04 Leverkusen,1,f24-22-2016-861392,False,True,Left to Right,Right to Left,Right to Left,Left to Right
3,2016-08-28T14:30:00,Hertha BSC,Sport-Club Freiburg,1,f24-22-2016-861393,False,False,Right to Left,Left to Right,Left to Right,Right to Left
4,2016-08-27T14:30:00,1. FC Köln,SV Darmstadt 98,1,f24-22-2016-861394,False,True,Right to Left,Left to Right,Left to Right,Right to Left
...,...,...,...,...,...,...,...,...,...,...,...
300,2016-10-30T16:30:00,1. FC Köln,Hamburger SV,9,f24-22-2016-861466,True,False,Right to Left,Left to Right,Left to Right,Right to Left
301,2016-10-29T14:30:00,FC Augsburg,FC Bayern München,9,f24-22-2016-861467,False,False,Left to Right,Right to Left,Right to Left,Left to Right
302,2016-10-29T14:30:00,SV Werder Bremen,Sport-Club Freiburg,9,f24-22-2016-861468,False,False,Right to Left,Left to Right,Left to Right,Right to Left
303,2016-10-29T14:30:00,SV Darmstadt 98,RB Leipzig,9,f24-22-2016-861469,False,False,Left to Right,Right to Left,Right to Left,Left to Right


In [4]:
opta_game_overview.Opta_Matchday.value_counts()

1     9
32    9
27    9
28    9
29    9
3     9
30    9
31    9
33    9
10    9
34    9
4     9
5     9
6     9
7     9
8     9
26    9
25    9
24    9
23    9
22    9
21    9
20    9
19    9
18    9
17    9
16    9
15    9
14    9
13    9
12    9
11    9
9     9
2     8
Name: Opta_Matchday, dtype: int64

## Overview of games in match information files

In [5]:
# relevant information collection lists

matchdays_numeric = []    # number of the matchday
DFL_mns = []    # dfl name of the match
home_teams =  []    # home teams
away_teams =  []    # away teams
dates = []           # date of the match
name_match = []      # does the match name in the file name and the match information match?!
end_scores = []       # end score of the match 




ROOT = os.getcwd() # returns current working directory of a process.

mi_data = (r"Matchinfo") # creates a raw string which treats backslash (\) as a literal character. 


for mi_file in os.listdir(mi_data):
    #print(mi_file)
    if mi_file.startswith('DFL_'):
        DFL_nm = mi_file[-18:-4]
        #print(DFL_nm)
        tree = etree.parse(str('/'.join([mi_data, mi_file])))
        root = tree.getroot()
        team_informations = root.find("MatchInformation").find("Teams")
        mi_home = root.find("MatchInformation").find("General").get("HomeTeamName")
        home_teams.append(mi_home)
        mi_away = root.find("MatchInformation").find("General").get("GuestTeamName")
        away_teams.append(mi_away)
        mi_mn = root.find("MatchInformation").find("General").get("MatchId")
        DFL_mns.append(mi_mn)
        mi_date = root.find("MatchInformation").find("General").get("PlannedKickoffTime")
        dates.append(mi_date)
        score = root.find("MatchInformation").find("General").get("Result")
        end_scores.append(score)
        mi_Matchday = root.find("MatchInformation").find("General").get("MatchDay")
        matchdays_numeric.append(mi_Matchday)
        boolean = DFL_nm == mi_mn
        name_match.append(boolean)
        
mi_game_overview = pd.DataFrame(list(zip(dates, home_teams, away_teams, end_scores, matchdays_numeric, DFL_mns, name_match)),
                             columns=['MI_Date', 'MI_Home', 'MI_Away', 'Final_Score', 'MI_Matchday', 'MI_Name', 'NameCorrect?'])

# mi_game_overview
mi_game_overview = mi_game_overview[:306] # remove relegation matches!
mi_game_overview

,MI_Date,MI_Home,MI_Away,Final_Score,MI_Matchday,MI_Name,NameCorrect?
0,2016-08-26T20:30:00.000+02:00,FC Bayern München,SV Werder Bremen,6:0,1,DFL-MAT-00277X,True
1,2016-08-27T15:30:00.000+02:00,Borussia Dortmund,1. FSV Mainz 05,2:1,1,DFL-MAT-00277Y,True
2,2016-08-27T18:30:00.000+02:00,Borussia Mönchengladbach,Bayer 04 Leverkusen,2:1,1,DFL-MAT-00277Z,True
3,2016-08-28T15:30:00.000+02:00,Hertha BSC,Sport-Club Freiburg,2:1,1,DFL-MAT-002780,True
4,2016-08-27T15:30:00.000+02:00,1. FC Köln,SV Darmstadt 98,2:0,1,DFL-MAT-002781,True
...,...,...,...,...,...,...,...
301,2017-05-20T15:30:00.000+02:00,1. FC Köln,1. FSV Mainz 05,2:0,34,DFL-MAT-0027GA,True
302,2017-05-20T15:30:00.000+02:00,Hamburger SV,VfL Wolfsburg,2:1,34,DFL-MAT-0027GB,True
303,2017-05-20T15:30:00.000+02:00,FC Ingolstadt 04,FC Schalke 04,1:1,34,DFL-MAT-0027GC,True
304,2017-05-20T15:30:00.000+02:00,TSG 1899 Hoffenheim,FC Augsburg,0:0,34,DFL-MAT-0027GD,True


In [6]:
mi_game_overview['MI_Matchday'].value_counts()
# top!

1     9
26    9
20    9
21    9
22    9
23    9
24    9
25    9
27    9
2     9
28    9
29    9
30    9
31    9
32    9
33    9
19    9
18    9
17    9
16    9
15    9
14    9
13    9
12    9
11    9
10    9
9     9
8     9
7     9
6     9
5     9
4     9
3     9
34    9
Name: MI_Matchday, dtype: int64

## Overview of games in position data files

In [7]:
# relevant information collection lists

matchdays_numeric = []    # number of the matchday
DFL_mns = []        # dfl name of the match




ROOT = os.getcwd() # returns current working directory of a process.

pos_data = (r"PositionData") # creates a raw string which treats backslash (\) as a literal character. 

for pos_file in os.listdir(pos_data):
    if pos_file.startswith('DFL_'):
        DFL_nm = pos_file[-18:-4]
        DFL_mns.append(DFL_nm)

                
pos_game_overview = pd.DataFrame(list(zip(DFL_mns)),
                             columns=['Pos_Name'])

pos_game_overview

# at least one missing
# 3 missing because 2 of these are relegation matches

,Pos_Name
0,DFL-MAT-00277X
1,DFL-MAT-00277Y
2,DFL-MAT-00277Z
3,DFL-MAT-002780
4,DFL-MAT-002781
...,...
300,DFL-MAT-0027GC
301,DFL-MAT-0027GD
302,DFL-MAT-0027GE
303,DFL-MAT-0027W0


In [8]:
dfl_overview = mi_game_overview
dfl_overview['Pos_Name'] = np.nan

for row in range(len(pos_game_overview)):
    name = pos_game_overview.Pos_Name[row]
    # print(name)
    if (mi_game_overview['MI_Name'].eq(name)).any():
        index = np.where(mi_game_overview.MI_Name == name)[0][0]
        # print(index)
        dfl_overview.loc[index, 'Pos_Name'] = name
    else:
        print(name)
    
dfl_overview

# DFL-MAT-0027W0 in pos but not mi = relegation
# DFL-MAT-0027W1 in pos but not mi = relegation

DFL-MAT-0027W0
DFL-MAT-0027W1


,MI_Date,MI_Home,MI_Away,Final_Score,MI_Matchday,MI_Name,NameCorrect?,Pos_Name
0,2016-08-26T20:30:00.000+02:00,FC Bayern München,SV Werder Bremen,6:0,1,DFL-MAT-00277X,True,DFL-MAT-00277X
1,2016-08-27T15:30:00.000+02:00,Borussia Dortmund,1. FSV Mainz 05,2:1,1,DFL-MAT-00277Y,True,DFL-MAT-00277Y
2,2016-08-27T18:30:00.000+02:00,Borussia Mönchengladbach,Bayer 04 Leverkusen,2:1,1,DFL-MAT-00277Z,True,DFL-MAT-00277Z
3,2016-08-28T15:30:00.000+02:00,Hertha BSC,Sport-Club Freiburg,2:1,1,DFL-MAT-002780,True,DFL-MAT-002780
4,2016-08-27T15:30:00.000+02:00,1. FC Köln,SV Darmstadt 98,2:0,1,DFL-MAT-002781,True,DFL-MAT-002781
...,...,...,...,...,...,...,...,...
301,2017-05-20T15:30:00.000+02:00,1. FC Köln,1. FSV Mainz 05,2:0,34,DFL-MAT-0027GA,True,DFL-MAT-0027GA
302,2017-05-20T15:30:00.000+02:00,Hamburger SV,VfL Wolfsburg,2:1,34,DFL-MAT-0027GB,True,DFL-MAT-0027GB
303,2017-05-20T15:30:00.000+02:00,FC Ingolstadt 04,FC Schalke 04,1:1,34,DFL-MAT-0027GC,True,DFL-MAT-0027GC
304,2017-05-20T15:30:00.000+02:00,TSG 1899 Hoffenheim,FC Augsburg,0:0,34,DFL-MAT-0027GD,True,DFL-MAT-0027GD


In [9]:
dfl_overview[80:90]
# 85 - 87
# DFL-MAT-0027AA HSV - BVB 10
# DFL-MAT-0027AB Ingolstadt - FCA 10
# DFL-MAT-0027AC Frankfurt - 1.FC


,MI_Date,MI_Home,MI_Away,Final_Score,MI_Matchday,MI_Name,NameCorrect?,Pos_Name
80,2016-10-30T15:30:00.000+01:00,TSG 1899 Hoffenheim,Hertha BSC,1:0,9,DFL-MAT-0027A5,True,DFL-MAT-0027A5
81,2016-11-05T15:30:00.000+01:00,FC Bayern München,TSG 1899 Hoffenheim,1:1,10,DFL-MAT-0027A6,True,DFL-MAT-0027A6
82,2016-11-05T15:30:00.000+01:00,Bayer 04 Leverkusen,SV Darmstadt 98,3:2,10,DFL-MAT-0027A7,True,DFL-MAT-0027A7
83,2016-11-06T17:30:00.000+01:00,FC Schalke 04,SV Werder Bremen,3:1,10,DFL-MAT-0027A8,True,DFL-MAT-0027A8
84,2016-11-04T20:30:00.000+01:00,Hertha BSC,Borussia Mönchengladbach,3:0,10,DFL-MAT-0027A9,True,DFL-MAT-0027A9
85,2016-11-05T15:30:00.000+01:00,Hamburger SV,Borussia Dortmund,2:5,10,DFL-MAT-0027AA,True,NaN
86,2016-11-05T15:30:00.000+01:00,FC Ingolstadt 04,FC Augsburg,0:2,10,DFL-MAT-0027AB,True,NaN
87,2016-11-05T18:30:00.000+01:00,Eintracht Frankfurt,1. FC Köln,1:0,10,DFL-MAT-0027AC,True,NaN
88,2016-11-05T15:30:00.000+01:00,Sport-Club Freiburg,VfL Wolfsburg,0:3,10,DFL-MAT-0027AD,True,DFL-MAT-0027AD
89,2016-11-06T15:30:00.000+01:00,RB Leipzig,1. FSV Mainz 05,3:1,10,DFL-MAT-0027AE,True,DFL-MAT-0027AE


In [10]:
dfl_overview.rename(columns={"MI_Home": "Home", "MI_Away": "Away", 'MI_Matchday': 'Matchday'}, inplace=True)
opta_game_overview.rename(columns={"Opta_Home": "Home", "Opta_Away": "Away", 'Opta_Matchday': 'Matchday'}, inplace=True)

game_overview = pd.merge(dfl_overview, opta_game_overview, on=["Home", "Away", "Matchday"], how='outer')
game_overview.to_csv('Output/GameOverview.csv')
game_overview

,MI_Date,Home,Away,Final_Score,Matchday,MI_Name,NameCorrect?,Pos_Name,Opta_Date,Opta_Name,Red_card,Retired,Direction_Home_HT1,Direction_Home_HT2,Direction_Away_HT1,Direction_Away_HT2
0,2016-08-26T20:30:00.000+02:00,FC Bayern München,SV Werder Bremen,6:0,1,DFL-MAT-00277X,True,DFL-MAT-00277X,2016-08-26T19:30:00,f24-22-2016-861390,False,False,Right to Left,Left to Right,Left to Right,Right to Left
1,2016-08-27T15:30:00.000+02:00,Borussia Dortmund,1. FSV Mainz 05,2:1,1,DFL-MAT-00277Y,True,DFL-MAT-00277Y,2016-08-27T14:30:00,f24-22-2016-861391,False,False,Left to Right,Right to Left,Right to Left,Left to Right
2,2016-08-27T18:30:00.000+02:00,Borussia Mönchengladbach,Bayer 04 Leverkusen,2:1,1,DFL-MAT-00277Z,True,DFL-MAT-00277Z,2016-08-27T17:30:00,f24-22-2016-861392,False,True,Left to Right,Right to Left,Right to Left,Left to Right
3,2016-08-28T15:30:00.000+02:00,Hertha BSC,Sport-Club Freiburg,2:1,1,DFL-MAT-002780,True,DFL-MAT-002780,2016-08-28T14:30:00,f24-22-2016-861393,False,False,Right to Left,Left to Right,Left to Right,Right to Left
4,2016-08-27T15:30:00.000+02:00,1. FC Köln,SV Darmstadt 98,2:0,1,DFL-MAT-002781,True,DFL-MAT-002781,2016-08-27T14:30:00,f24-22-2016-861394,False,True,Right to Left,Left to Right,Left to Right,Right to Left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,2017-05-20T15:30:00.000+02:00,1. FC Köln,1. FSV Mainz 05,2:0,34,DFL-MAT-0027GA,True,DFL-MAT-0027GA,2017-05-20T14:30:00,f24-22-2016-861691,False,False,Right to Left,Left to Right,Left to Right,Right to Left
302,2017-05-20T15:30:00.000+02:00,Hamburger SV,VfL Wolfsburg,2:1,34,DFL-MAT-0027GB,True,DFL-MAT-0027GB,2017-05-20T14:30:00,f24-22-2016-861692,False,False,Right to Left,Left to Right,Left to Right,Right to Left
303,2017-05-20T15:30:00.000+02:00,FC Ingolstadt 04,FC Schalke 04,1:1,34,DFL-MAT-0027GC,True,DFL-MAT-0027GC,2017-05-20T14:30:00,f24-22-2016-861693,True,False,Left to Right,Right to Left,Right to Left,Left to Right
304,2017-05-20T15:30:00.000+02:00,TSG 1899 Hoffenheim,FC Augsburg,0:0,34,DFL-MAT-0027GD,True,DFL-MAT-0027GD,2017-05-20T14:30:00,f24-22-2016-861694,False,False,Right to Left,Left to Right,Left to Right,Right to Left


In [ ]:
game_overview[250:306]
# SV Werder Bremen	FC Augsburg	2	DFL-MAT-00278B	 missing event data?!

# Missing Data!

### EventData
Matchday 2 - SV Werder Bremen vs FC Augsburg - DFL_Name = DFL-MAT-00278B

### PositionData
Matchday 10 - Hamburger SV vs. Borussia Dortmund - DFL_Name = DFL-MAT-0027AA <br>
Matchday 10 - FC Ingolstadt 04 vs. FC Augsburg - DFL_Name = DFL-MAT-0027AB <br>
Matchday 10 - Eintracht Frankfurt vs. 1. FC Köln - DFL_Name = DFL-MAT-0027AC <br>

In [11]:
# exclude missing data

game_overview[game_overview.Pos_Name.isna()]
game_overview[game_overview.Opta_Name.isna()]

game_overview2 = game_overview[game_overview.Pos_Name.notna() & game_overview.Opta_Name.notna()].reset_index()
game_overview2.to_csv('Output/GameOverview2.csv')
game_overview2

,index,MI_Date,Home,Away,Final_Score,Matchday,MI_Name,NameCorrect?,Pos_Name,Opta_Date,Opta_Name,Red_card,Retired,Direction_Home_HT1,Direction_Home_HT2,Direction_Away_HT1,Direction_Away_HT2
0,0,2016-08-26T20:30:00.000+02:00,FC Bayern München,SV Werder Bremen,6:0,1,DFL-MAT-00277X,True,DFL-MAT-00277X,2016-08-26T19:30:00,f24-22-2016-861390,False,False,Right to Left,Left to Right,Left to Right,Right to Left
1,1,2016-08-27T15:30:00.000+02:00,Borussia Dortmund,1. FSV Mainz 05,2:1,1,DFL-MAT-00277Y,True,DFL-MAT-00277Y,2016-08-27T14:30:00,f24-22-2016-861391,False,False,Left to Right,Right to Left,Right to Left,Left to Right
2,2,2016-08-27T18:30:00.000+02:00,Borussia Mönchengladbach,Bayer 04 Leverkusen,2:1,1,DFL-MAT-00277Z,True,DFL-MAT-00277Z,2016-08-27T17:30:00,f24-22-2016-861392,False,True,Left to Right,Right to Left,Right to Left,Left to Right
3,3,2016-08-28T15:30:00.000+02:00,Hertha BSC,Sport-Club Freiburg,2:1,1,DFL-MAT-002780,True,DFL-MAT-002780,2016-08-28T14:30:00,f24-22-2016-861393,False,False,Right to Left,Left to Right,Left to Right,Right to Left
4,4,2016-08-27T15:30:00.000+02:00,1. FC Köln,SV Darmstadt 98,2:0,1,DFL-MAT-002781,True,DFL-MAT-002781,2016-08-27T14:30:00,f24-22-2016-861394,False,True,Right to Left,Left to Right,Left to Right,Right to Left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,301,2017-05-20T15:30:00.000+02:00,1. FC Köln,1. FSV Mainz 05,2:0,34,DFL-MAT-0027GA,True,DFL-MAT-0027GA,2017-05-20T14:30:00,f24-22-2016-861691,False,False,Right to Left,Left to Right,Left to Right,Right to Left
298,302,2017-05-20T15:30:00.000+02:00,Hamburger SV,VfL Wolfsburg,2:1,34,DFL-MAT-0027GB,True,DFL-MAT-0027GB,2017-05-20T14:30:00,f24-22-2016-861692,False,False,Right to Left,Left to Right,Left to Right,Right to Left
299,303,2017-05-20T15:30:00.000+02:00,FC Ingolstadt 04,FC Schalke 04,1:1,34,DFL-MAT-0027GC,True,DFL-MAT-0027GC,2017-05-20T14:30:00,f24-22-2016-861693,True,False,Left to Right,Right to Left,Right to Left,Left to Right
300,304,2017-05-20T15:30:00.000+02:00,TSG 1899 Hoffenheim,FC Augsburg,0:0,34,DFL-MAT-0027GD,True,DFL-MAT-0027GD,2017-05-20T14:30:00,f24-22-2016-861694,False,False,Right to Left,Left to Right,Left to Right,Right to Left


In [15]:
game_names_Opta = {game_overview2.Opta_Name[i]: game_overview2.MI_Name[i] for i in range(len(game_overview2.Opta_Name))}

with open("Output/game_names_Opta.pkl", "wb") as cdict:
    pickle.dump(game_names_Opta, cdict)
print(game_names_Opta)

game_MD_Opta = {game_overview2.Opta_Name[i]: game_overview2.Matchday[i] for i in range(len(game_overview2.Opta_Name))}
print(game_MD_Opta)

with open("Output/game_MD_Opta.pkl", "wb") as cdict:
    pickle.dump(game_MD_Opta, cdict)



{'f24-22-2016-861390': 'DFL-MAT-00277X', 'f24-22-2016-861391': 'DFL-MAT-00277Y', 'f24-22-2016-861392': 'DFL-MAT-00277Z', 'f24-22-2016-861393': 'DFL-MAT-002780', 'f24-22-2016-861394': 'DFL-MAT-002781', 'f24-22-2016-861395': 'DFL-MAT-002782', 'f24-22-2016-861396': 'DFL-MAT-002783', 'f24-22-2016-861397': 'DFL-MAT-002784', 'f24-22-2016-861398': 'DFL-MAT-002785', 'f24-22-2016-861399': 'DFL-MAT-002786', 'f24-22-2016-861400': 'DFL-MAT-002787', 'f24-22-2016-861401': 'DFL-MAT-002788', 'f24-22-2016-861402': 'DFL-MAT-002789', 'f24-22-2016-861403': 'DFL-MAT-00278A', 'f24-22-2016-861405': 'DFL-MAT-00278C', 'f24-22-2016-861406': 'DFL-MAT-00278D', 'f24-22-2016-861407': 'DFL-MAT-00278E', 'f24-22-2016-861408': 'DFL-MAT-00278F', 'f24-22-2016-861409': 'DFL-MAT-00278G', 'f24-22-2016-861410': 'DFL-MAT-00278H', 'f24-22-2016-861411': 'DFL-MAT-00278I', 'f24-22-2016-861412': 'DFL-MAT-00278J', 'f24-22-2016-861413': 'DFL-MAT-00278K', 'f24-22-2016-861414': 'DFL-MAT-00278L', 'f24-22-2016-861415': 'DFL-MAT-00278M',

In [ ]:
game_names_DFL = {game_overview2.MI_Name[i]: game_overview2.Opta_Name[i] for i in range(len(game_overview2.MI_Name))}

with open("Output/game_names_DFL.pkl", "wb") as cdict:
    pickle.dump(game_names_DFL, cdict)
game_names_DFL